# 0.1: Compile

In this example, we compile our simple unified Ivy function `normalize` from the last section. We then show how this newly compiled `normalize` function exhibits much better runtime performance than the non-compiled version.

Firstly, let's pick up where we left off in the last notebook, with our unified `normalize` function:

In [ ]:
import ivy
import torch

def normalize(x, mean, std):
    return torch.div(torch.sub(x, mean), std)

normalize = ivy.unify(normalize)

For the purpose of illustration, we will use JAX:

In [ ]:
# set Ivy's backend to JAX
ivy.set_backend("jax")

# Import Jax NumPy API
import jax.numpy as jnp

# create random NumPy arrays for testing
x = jnp.randon.uniform(size=10)
mean = jnp.mean(x)
std = jnp.std(x)

As in the previous example, the unified function can be executed like so (in this case it will trigger lazy unification, see the [Lazy vs Eager]() section for more details):

In [ ]:
normalize(x, mean, std)

When calling this function, all of Ivy's function wrapping is included in the call stack of `normalize`, which adds runtime overhead. In general, `ivy.compile` strips any arbitrary function down to its constituent functions in the functional API of the target framework. It will then also be compiled to machine-code if the target framework supports low-level compiling (via functions such as `tf.function`, `torch.jit.script`, `torch.jit.trace`, `torch.compile`, `jax.jit` etc.). The code can be compiled like so:

In [ ]:
comp = ivy.compile(normalize)  # compiles to JAX, due to ivy.set_backend

The compiled function can be executed in exactly the same manner as the non-compiled function (in this case it will also trigger lazy compilation, see the [Lazy vs Eager]() section for more details):

In [ ]:
comp(x, mean, std)

The machine-code compilation can be turned off by setting the argument `low_level = False`, in which case it will simply return a chain of Python function in the functional API of the target framework (in this case JAX). This will still improve the runtime efficiency over the original un-compiled version due to the removal of all Ivy wrapping overhead, but it will not be as runtime efficient as the low-level compiled version:

In [ ]:
partial_comp = ivy.compile(normalize, low_level=False)

Again, the compiled function can be executed in exactly the same manner as the non-compiled function (in this case it will also trigger lazy compilation, see the [Lazy vs Eager]() section for more details):

In [ ]:
partial_comp(x, mean, std)

With all lazy unification and compilation calls now performed (which all increase runtime during the very first call of the function), we can now assess the runtime efficiencies of each function:

In [ ]:
ivy.time_function(normalize)(x, mean, std)
ivy.time_function(partial_comp)(x, mean, std)
ivy.time_function(comp)(x, mean, std)

As expected, we can see that the slowest is `normalize`, which includes all Ivy wrapping overhead. Next is `partial_comp` which has no wrapping overhead but is still expressed entirely in Python, without compiling to low-level code. The fastest is `comp` because the wrapping overhead is removed and the function is compiled to low-level code for maximal efficiency.

## Round Up

That's it, you can now compile Ivy code for more efficient inference! However, there are several other important topics to master before you're ready to unify ML code like a pro 🥷. Next, we'll be learning how to transpile code from one framework to another in a single line of code 🔄